In [72]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url) # Getting docs
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [73]:
# RAG with Vector Search

# qdrant and fastembed are already installed using pip
# qdrant client is running using docker

from qdrant_client import QdrantClient, models

qd_client = QdrantClient("http://localhost:6333")

EMBEDDING_DIMENSIONALITY = 512

model_handle = "jinaai/jina-embeddings-v2-small-en"

In [74]:
collection_name = "hw_2"

qd_client.delete_collection(collection_name)

# Create collection
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [75]:
points = []
id = 0

for course in documents_raw:
    for doc in course["documents"]:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc["text"], model=model_handle), # embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"]
            }
        )
        points.append(point)

        id += 1

In [10]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [76]:
# Search for docs
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [77]:
question = "I just discovered the course. Can I join now?"

results = search(question, limit=1)

results

QueryResponse(points=[])

In [8]:
from fastembed import TextEmbedding
import numpy as np

embed_model = TextEmbedding(model_handle)

query_embedding = list(embed_model.embed([question]))[0]

In [ ]:
np.linalg.norm(query_embedding)

np.float64(1.0)

In [ ]:
query_embedding.dot(query_embedding)

np.float64(1.0000000000000002)

## Q1. Embedding the query

Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

-0.51
-0.11
0
0.51

In [11]:
np.min(query_embedding)

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

Now let's embed this document:

doc = 'Can I still join the course after the start date?'

What's the cosine similarity between the vector for the query and the vector for the document?

0.3
0.5
0.7
0.9

In [12]:
question_doc = 'Can I still join the course after the start date?'
query_embedding_doc = list(embed_model.embed([question_doc]))[0]

In [13]:
query_embedding.dot(query_embedding_doc)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

0
1
2
3
4

In [43]:
q3_4_documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
texts = []
for doc in q3_4_documents:
    texts.append(doc['text'])

In [47]:
query_embedding_texts = list(embed_model.embed(texts))

In [50]:
np.dot(query_embedding_texts, query_embedding)

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

Document 0 Similarity: 0.76296847
Document 1 Similarity: 0.81823782
Document 2 Similarity: 0.80853974
Document 3 Similarity: 0.7133079
Document 4 Similarity: 0.73044992

So ranking with the highest similarity is Document 1

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

full_text = doc['question'] + ' ' + doc['text']
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

0
1
2
3
4
Is it different from Q3? If yes, why?

In [51]:
full_texts = []
for doc in q3_4_documents:
    full_texts.append(doc['question'] + ' ' + doc['text'])

In [52]:
query_embedding_full_texts = list(embed_model.embed(full_texts))

In [53]:
np.dot(query_embedding_full_texts, query_embedding)

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

Document 0 Similarity = 0.85145432
Document 1 Similarity = 0.84365942
Document 2 Similarity = 0.8408287
Document 3 Similarity = 0.7755158
Document 4 Similarity = 0.80860078

## Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

128
256
384
512
One of these models is BAAI/bge-small-en. Let's use it.

In [ ]:
supported_models = TextEmbedding.list_supported_models()

for model in supported_models:
    model_name = model['model']
    dim = model['dim']
    print(f"Model: {model_name}, Dimension: {dim}")

Model: BAAI/bge-base-en, Dimension: 768
Model: BAAI/bge-base-en-v1.5, Dimension: 768
Model: BAAI/bge-large-en-v1.5, Dimension: 1024
Model: BAAI/bge-small-en, Dimension: 384
Model: BAAI/bge-small-en-v1.5, Dimension: 384
Model: BAAI/bge-small-zh-v1.5, Dimension: 512
Model: mixedbread-ai/mxbai-embed-large-v1, Dimension: 1024
Model: snowflake/snowflake-arctic-embed-xs, Dimension: 384
Model: snowflake/snowflake-arctic-embed-s, Dimension: 384
Model: snowflake/snowflake-arctic-embed-m, Dimension: 768
Model: snowflake/snowflake-arctic-embed-m-long, Dimension: 768
Model: snowflake/snowflake-arctic-embed-l, Dimension: 1024
Model: jinaai/jina-clip-v1, Dimension: 768
Model: Qdrant/clip-ViT-B-32-text, Dimension: 512
Model: sentence-transformers/all-MiniLM-L6-v2, Dimension: 384
Model: jinaai/jina-embeddings-v2-base-en, Dimension: 768
Model: jinaai/jina-embeddings-v2-small-en, Dimension: 512
Model: jinaai/jina-embeddings-v2-base-de, Dimension: 768
Model: jinaai/jina-embeddings-v2-base-code, Dimension

Smallest model is of 384 dimensions:

Model: BAAI/bge-small-en, Dimension: 384
Model: BAAI/bge-small-en-v1.5, Dimension: 384
Model: snowflake/snowflake-arctic-embed-xs, Dimension: 384
Model: snowflake/snowflake-arctic-embed-s, Dimension: 384
Model: sentence-transformers/all-MiniLM-L6-v2, Dimension: 384
Model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2, Dimension: 384

# Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

text = doc['question'] + ' ' + doc['text']

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record):

0.97
0.87
0.77
0.67

In [ ]:
ml_zoomcamp_documents = []
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue
        
    for doc in course['documents']:
        doc['course'] = course_name
        ml_zoomcamp_documents.append(doc)

print(f"Num of docs: {len(ml_zoomcamp_documents)}")

Num of docs: 375


In [68]:
smaller_model_handle = "BAAI/bge-small-en"
smaller_embedding_model = TextEmbedding(smaller_model_handle)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [69]:
smaller_embedding_dimension = 384

In [ ]:
collection_name_q6 = "hw2-q6"

qd_client.delete_collection(collection_name_q6)
qd_client.create_collection(
    collection_name=collection_name_q6,
    vectors_config=models.VectorParams(
        size=smaller_embedding_dimension,
        distance=models.Distance.COSINE
    )
)

True

In [ ]:
points = []
id = 0

for doc in ml_zoomcamp_documents:
    full_text = doc['question'] + ' ' + doc['text']
    
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=full_text, model=smaller_model_handle),
        payload={
            "text": doc['text'],
            "question": doc['question'],
            "full_text": full_text,
            "course": doc['course']
        }
    )
    points.append(point)
    id += 1

In [81]:
qd_client.upsert(
    collection_name=collection_name_q6,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [82]:
q6_query_text = "I just discovered the course. Can I join now?"

q6_results = qd_client.query_points(
    collection_name=collection_name_q6,
    query=models.Document(
        text=q6_query_text,
        model=smaller_model_handle
    ),
    limit=1,  # Get top 1 results to see all scores
    with_payload=True
)

In [84]:
print(f"Highest score: {q6_results.points[0].score}")

Highest score: 0.8703172
